<a href="https://colab.research.google.com/github/deanhoperobertson/NLP-Text-Classifiation-/blob/master/Text_Classification_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Amazon Reviews 

In [0]:
import urllib.request
import pandas as pd
import urllib
from sklearn import model_selection, preprocessing

#Feature Engineering
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
#import data from my github repo
url = "https://raw.githubusercontent.com/deanhoperobertson/NLP-Text-Classifiation-/master/corpus"
data = urllib.request.urlopen(url).read()

In [0]:
labels, texts = [], []
for i, line in enumerate(str(data).split("\\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# create a dataframe using texts and lables
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

In [27]:
trainDF.head()

,text,label
0,Stuning even for the non-gamer: This sound tra...,b'__label__2
1,The best soundtrack ever to anything.: I\'m re...,__label__2
2,Amazing!: This soundtrack is my favorite music...,__label__2
3,Excellent Soundtrack: I truly like this soundt...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2


In [0]:
#split into train and test set (default is 25%)
#set seed for consistency
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size=0.25, random_state=1)


In [0]:
# encode target labels into intergers between 0 - no.classes
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

## Feature Engineeering

Now we need to convert the worded sentences into vectors by replacing the words with numercil representation. 

for Example:
"The cat sat on the hat" = {1,3,10,7,1,12}

This can be done in a number of ways such as:
-  CountVectorizer (Bag of Words)
- TF-IDF Matrix



### CountVectorizer

In [0]:
# instantiate a count vectorizer object to tokenize on a word level
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', strip_accents ="ascii")
count_vect.fit(trainDF['text'])

#see the unique words being tokenized 
count_vect.get_feature_names()[1:10]

#create a document term matrix using the 
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [32]:
#view that DT matrix in dataframe
pd.DataFrame(xtrain_count.toarray(), columns=count_vect.get_feature_names()).head()

,0,00,000,001,002,00290,007,0070412901,0072316373,008,...,zzzz,zzzzz,zzzzzz,zzzzzzz,zzzzzzzzzz,zzzzzzzzzzzz,zzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
